# #05. Lazy Predict

## 1. Modules import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from random import randint, sample

from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, recall_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

from codecarbon import EmissionsTracker

In [2]:
#!pip install lazypredict

#### Importing our modules:

In [3]:
from teda import TEDA
from RLSFilter import RLSFilter

from treating import dataTreating, dataSpliting, dataProcessing, dataSplit

In [4]:
def find_indices(list_to_check, item_to_find):
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == item_to_find:
            indices.append(idx)
    return indices

## 2. Data managing

### 2.1 Power data:

In [5]:
data = pd.read_csv("Electric_Production.csv")
data.rename(columns={'DATE':'Date', 'IPG2211A2N':'Production'}, inplace=True)
data.head()

#Variables
series = "Production"
date = "Date"

### 2.2 Inserting outliers:

In [6]:
data['label'] = 0

In [7]:
outliers_percentage = 8/100

#Outliers do tipo zero
#z = o[0:np.round(len(o)/2).astype(int)]
z = [61,37,287,230,214,162,98,30,278,28,25,314,348,195,113,387]
#Outliers do tipo pico
#p = o[np.round(len(o)/2).astype(int): -1]
p = [125,242,291,377,34,206,249,118,389,12,376,375,74,146,272,368]

#### Peak-type outliers will have **twice** the magnitude. Therefore:

In [8]:
data['Outlier'] = data[series]

data['Outlier'].iloc[p] = 1.5*data[series].iloc[p]
data['label'].iloc[p] = 1

c:\Users\Samsung\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


#### Zero-type outliers has value equals **zero**:

In [9]:
data['Outlier'].iloc[z] = 0
data['label'].iloc[z] = 1

### Let's see the new curve:

In [10]:
figure = make_subplots(specs=[[{"secondary_y": True}]])

figure.update_layout(
    title="Dataset with outliers"
)

figure.add_trace(
    go.Scatter(y=data['Outlier'], name='With Outliers'),
    secondary_y=False
)

figure.update_xaxes(title='Time (months)')
figure.update_yaxes(title='Eletric Production', secondary_y=False)
figure.show()

## Returning the RMSE values ​​of the other algorithms:

In [11]:
rmse_tf_pred = 15.571152
rmse_lstm_pred = 18.98823	
rmse_knn_pred = 22

In [12]:
dict_models = {
    'TEDA-Forecasting': rmse_tf_pred,
    'LSTM': rmse_lstm_pred,
    'kNN': rmse_knn_pred,
}
df_models = pd.DataFrame.from_dict(dict_models, orient='index', columns=['RMSE']).reset_index().rename(columns={'index':'Model'})
df_models

,Model,RMSE
0,TEDA-Forecasting,15.571152
1,LSTM,18.988230
2,kNN,22.000000


# 3. LazyPredict

### 3.1 Importing modules:

In [13]:
import lazypredict
from lazypredict.Supervised import LazyRegressor

### 3.2 Defining model:

In [14]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)

In [16]:
# Data processing
df, df_train, df_test, X_train, y_train, X_test, y_test = dataSplit(data["Outlier"], data["label"], window=4, forecasting=4, N_splits=4)

In [17]:
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

  5%|▍         | 2/41 [00:00<00:02, 19.01it/s]

AdaBoostRegressor model failed to execute
y should be a 1d array, got an array of shape (312, 4) instead.
BayesianRidge model failed to execute
y should be a 1d array, got an array of shape (312, 4) instead.


 15%|█▍        | 6/41 [00:00<00:01, 19.56it/s]

ElasticNetCV model failed to execute
For multi-task outputs, use MultiTaskElasticNetCV


 22%|██▏       | 9/41 [00:00<00:02, 12.77it/s]

...

NuSVR model failed to execute
y should be a 1d array, got an array of shape (312, 4) instead.
OrthogonalMatchingPursuitCV model failed to execute
y should be a 1d array, got an array of shape (312, 4) instead.
PassiveAggressiveRegressor model failed to execute
y should be a 1d array, got an array of shape (312, 4) instead.
PoissonRegressor model failed to execute
y should be a 1d array, got an array of shape (312, 4) instead.


 95%|█████████▌| 39/41 [00:02<00:00, 19.33it/s]

SGDRegressor model failed to execute
y should be a 1d array, got an array of shape (312, 4) instead.
SVR model failed to execute
y should be a 1d array, got an array of shape (312, 4) instead.
TweedieRegressor model failed to execute
y should be a 1d array, got an array of shape (312, 4) instead.


100%|██████████| 41/41 [00:03<00:00, 12.43it/s]

LGBMRegressor model failed to execute
y should be a 1d array, got an array of shape (312, 4) instead.


In [18]:
df_lazy = pd.DataFrame(data=models.RMSE[0:-2].values, index=models.RMSE[0:-2].index, columns=['RMSE']).reset_index()
df_lazy

,Model,RMSE
0,Lars,24.37
1,TransformedTargetRegressor,24.37
2,LinearRegression,24.37
3,Ridge,24.37
4,RidgeCV,24.40
5,Lasso,24.71
6,RandomForestRegressor,25.00
7,ElasticNet,25.07
8,KNeighborsRegressor,25.19
9,BaggingRegressor,25.70


## Creating DataFrame with all RMSE models

In [20]:
df_all = df_models.append(df_lazy).reset_index(drop=True)


In [21]:
RMSE_label = df_all['Model']
RMSE_value = df_all['RMSE']

In [22]:
fig, ax = plt.subplots(figsize = (28,10), dpi=500)

p1 = ax.bar(RMSE_label, RMSE_value)
ax.bar_label(p1, label_type='edge', fmt='%.2f', fontsize=12)
p1[0].set_color('green')
p1[1].set_color('red')
p1[2].set_color('red')

plt.title("Models comparison", fontsize=20)
plt.xlabel("Type", fontsize=20)
plt.ylabel("Root mean squared error (RMSE)", fontsize=20)
plt.xticks(fontsize=15, rotation=90)
plt.yticks(fontsize=15, rotation=90)

(array([ 0., 10., 20., 30., 40., 50., 60.]),
 [Text(0, 0.0, '0'),
  Text(0, 10.0, '10'),
  Text(0, 20.0, '20'),
  Text(0, 30.0, '30'),
  Text(0, 40.0, '40'),
  Text(0, 50.0, '50'),
  Text(0, 60.0, '60')])

# 5. References

 [[1]](https://www.mdpi.com/1424-8220/22/10/3838) **Andrade, P.**; Silva, I.; Silva, M.; Flores, T.; Cassiano, J.; Costa, D.G. *A TinyML Soft-Sensor Approach for Low-Cost Detection and Monitoring of Vehicular Emissions*. SENSORS 2022, 22, 3838.  ![GitHub](https://img.shields.io/badge/DOI-10.3390%2Fs22103838-green)


[[2]](https://dl.acm.org/journal/tecs) **Andrade, P.**; Silva, I.; Silva, M.; Flores, T.; Costa, D.G. Soares, E.; _Online Processing of Vehicular Data on the Edge Through an Unsupervised TinyML Regression Technique_. ACM TECS 2023. ![GitHub](https://img.shields.io/badge/DOI-10.1145%2Fs3591356-green)